In [ ]:
### Import the libraries

# !pip install spotipy
import spotipy
import sys
import json

import numpy as np
import pandas as pd

from spotipy.oauth2 import SpotifyClientCredentials


In [ ]:
# Initialize parameters

cid="XXX"
csecret="XXX"
user_id = "XXX"
plist_id = "XXX"


In [ ]:
# Connect to Spotify to obtain the ids of of tracks in the playlist being scapred

client_credentials_manager = SpotifyClientCredentials(client_id=cid,
                                                      client_secret=csecret)

spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
 
#get the track list information for the songs in the playlist
response = spotify.user_playlist(user = user_id,
                                 playlist_id = plist_id,
                                 fields="tracks,next")
tracks=response["tracks"] 

# get the tracksids : 100 songs
trackids=[i["track"]["uri"].split(":")[2] for i in tracks["items"] ]
 

len(trackids)

In [ ]:
# keep on looping to the next songs until end of the playlist
while tracks['next']: 
    tracks=spotify.next(tracks)
    trackids.extend([i["track"]["uri"].split(":")[2] for i in tracks["items"] ])

len(trackids)

In [ ]:
#get the features we would like to keep
extract_features=np.array(list(spotify.audio_features(trackids[5])[0].keys()))
extract_features

In [ ]:
#get song titles & song features
title=[]
song_features=[]

j=0
for i in trackids:
    j=j+1
    
    #title update
    title.append(spotify.track(i)["name"])
    #song update:
    song_features.append([spotify.audio_features(i)[0] [x] for x in extract_features])
    
    
    if j%50==0:
        print(j)

In [ ]:
#get artist names & song feature
artist1=[]
artist2=[]


j=0
for i in trackids:
    j=j+1
    
    
    # artist update
    artist1.append(spotify.track(i)["artists"][0]["name"])
    if len(spotify.track(i)["artists"])==1:
        artist2.append(np.nan)
    else:
        artist2.append(spotify.track(i)["artists"][1]["name"])
        if j%50==0:
            print(j)

In [ ]:
#convert the information to pandas dataframe
Playlist=pd.DataFrame(song_features)
Playlist.columns=extract_features
Playlist=Playlist.assign(Title=title,Artist1=artist1,Artist2=artist2)
Playlist